## Telegram Scraping and Collection Pipeline

In [3]:
import logging
import json
import nest_asyncio
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

# Apply nest_asyncio to avoid event loop conflicts in Jupyter
nest_asyncio.apply()

# Configure logging
logging.basicConfig(filename='scraping.log', level=logging.INFO, format='%(asctime)s - %(message)s')

# Use your own values from my.telegram.org
api_id = "28822767"  # Replace with your actual API ID
api_hash = '6b5b1a99af22f4c184d5a7acd8a7c27f'  # Replace with your actual API Hash

# Create the Telegram client
client = TelegramClient('telegram_scrap', api_id, api_hash)

# Connect to Telegram
await client.connect()

# Check if already authenticated
if not await client.is_user_authorized():
    phone_number = input("Enter your phone number (with country code): ")
    await client.send_code_request(phone_number)
    code = input("Enter the verification code you received: ")

    try:
        await client.sign_in(phone_number, code)
    except SessionPasswordNeededError:
        password = input("Enter your Telegram password: ")  # Two-step verification password
        await client.sign_in(password=password)

# List of channels to scrape
channels = [
    'https://t.me/DoctorsET',
    'https://t.me/lobelia4cosmetics',
    'https://t.me/yetenaweg',
    'https://t.me/EAHCI'
]

# Store messages
data = []

# Iterate over messages in each channel
for channel in channels:
    async for message in client.iter_messages(channel):
        raw_message = {
            'channel': channel,
            'message_id': message.id,
            'date': str(message.date),
            'sender_id': message.sender_id,
            'message': message.text
        }
        data.append(raw_message)

        # Log message
        logging.info(f"Message from {channel}: {message.text}")

# Save data to a JSON file
with open('scraped_data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print("✅ Scraping completed. Data saved to 'scraped_data.json'.")

# Disconnect the client after scraping
await client.disconnect()


✅ Scraping completed. Data saved to 'scraped_data.json'.
